# LiDAR の取得

Matplotlib のインストールを行うコマンドが入っています。一度実行した後はこの処理はスキップ可能です。初回インストール後にkernelの再起動が必要です。

In [ ]:
!pip install -q matplotlib

In [ ]:
%run setup_demo_env.py

In [ ]:
import math

import grpc
import kachaka_api_pb2
import matplotlib.patches as patches
import matplotlib.pyplot as plt
from IPython.display import Image, clear_output
from kachaka_api_pb2_grpc import KachakaApiStub

stub = KachakaApiStub(grpc.aio.insecure_channel(kachaka_api_server))

stub.SetManualControlEnabled(
    kachaka_api_pb2.SetManualControlEnabledRequest(enable=True)
)

req = kachaka_api_pb2.GetRequest()
while True:
    resp = await stub.GetRosLaserScan(req)
    req.metadata.cursor = resp.metadata.cursor

    clear_output(wait=True)
    fig = plt.figure(figsize=(5, 5))

    n = len(resp.scan.ranges)
    x = list(range(n))
    y = list(range(n))
    for i in range(n):
        theta = resp.scan.angle_min + resp.scan.angle_increment * i
        x[i] = resp.scan.ranges[i] * math.cos(theta)
        y[i] = resp.scan.ranges[i] * math.sin(theta)

    plt.plot(0, 0, "o", color="black")
    plt.plot(x, y, ".")
    plt.xlim(-6.0, 6.0)
    plt.ylim(-6.0, 6.0)
    plt.grid(True)
    plt.gca().set_aspect("equal", adjustable="box")
    plt.show()